# Othello

Othello is a turn-based two-player strategy board game. The players take turns placing pieces, one player white and the other player black, on an 8x8 board in such a way that captures some of the opponent's pieces, with the goal of finishing the game with more pieces of their color on the board.

Every move must capture one or more of the opponent's pieces. To capture, player A places a piece adjacent to one of player B's pieces so that there is a straight line (horizontal, vertical or diagonal) of adjacent pieces that begins with one of player A's pieces, continues with one or more of player B's pieces, and ends with one of player A's pieces.

For example, if Black places a piece on square (5, 1), he will capture all of Black's pieces (5, 1) and (5, 6):

<img src='files/img/capture.png'>

For more information about the game (which is also known as Reversi) including detailed rules, see the entry on [Wikipedia](https://en.wikipedia.org/wiki/Reversi). Additionally, this implementation doesn't take into account some tournament-style Othello details, such as game time limits and a different indexing scheme.

We will implement representations for the board and pieces and the mechanics of playing a game. We will then explore several game-playing strategies. There is a simple command-line program provided for playing against the computer or comparing two strategies.

Written by [Daniel Connelly](http://dhconnelly.com). This implementation follows chapter 18 of Peter Norvig's "Paradigms of Artificial Intelligence".

## Table of contents

1. [Board representation](#board)
2. [Playing the game](#playing)
3. [Strategies](#strategies)
   - [Random](#random)<br>
   - [Local maximization](#localmax)<br>
   - [Minimax search](#minimax)<br>
   - [Alpha-beta search](#alphabeta)<br>
4. [Conclusion](#conclusion)

<a id="board"></a>

## Board Representation

We represent the board as a 100-element list, which includes each square on the board as well as the outside edge. Each consecutive sublist of ten elements represents a single row, and each list element stores a piece. An initial board contains four pieces in the center:

```
     ? ? ? ? ? ? ? ? ? ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . o @ . . . ?
     ? . . . @ o . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? ? ? ? ? ? ? ? ? ?
```

This representation has two useful properties:
1. Square (m, n) can be accessed as `board[mn]`. This avoids the need to write functions that convert between square locations and list indexes.
2. Operations involving bounds checking are slightly simpler.

The outside edge is marked `?`, empty squares are `.`, black is `@`, and white is `o`. The black and white pieces represent the two players.

In [1]:
EMPTY, BLACK, WHITE, OUTER = '.', '@', 'o', '?'
PIECES = (EMPTY, BLACK, WHITE, OUTER)
PLAYERS = {BLACK: 'Black', WHITE: 'White'}

To refer to neighbor squares we can add a direction to a square.

In [2]:
UP, DOWN, LEFT, RIGHT = -10, 10, -1, 1
UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, UP_LEFT = -9, 11, 9, -11
DIRECTIONS = (UP, UP_RIGHT, RIGHT, DOWN_RIGHT, DOWN, DOWN_LEFT, LEFT, UP_LEFT)

In [3]:
def squares():
    """List all the valid squares on the board"""
    return [i for i in range(11, 89) if 1 <= (i % 10) <= 8]

def initial_board():
    """Create a new board with the initial black and white positions filled"""
    board = [OUTER] * 100
    for i in squares():
        board[i] = EMPTY
    # The middle four squares should hold the initial piece positions.
    board[44], board[45] = WHITE, BLACK
    board[54], board[55] = BLACK, WHITE
    return board

def print_board(board):
    """Get a string representation of the board."""
    rep = ''
    rep += '  {0}\n'.format(' '.join(map(str, range(1, 9))))
    for row in range(1, 9):
        begin, end = 10*row + 1, 10*row + 9
        rep += '{0} {1}\n'.format(row, ' '.join(board[begin:end]))
    return rep

print(print_board(initial_board()))

  1 2 3 4 5 6 7 8
1 . . . . . . . .
2 . . . . . . . .
3 . . . . . . . .
4 . . . o @ . . .
5 . . . @ o . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .



<a id="playing"></a>

## Playing the Game

We need functions to get moves from players, check to make sure that the moves are legal, apply the moves to the board, and detect when the game is over.

### Checking Moves

A move must be both valid and legal: it must refer to a real square, and it must form a bracket with another piece of the same color with pieces of the opposite color in between.

In [4]:
def is_valid(move):
    """Is move a square on the board?"""
    return isinstance(move, int) and move in squares()

def opponent(player):
    """Get player's opponent piece."""
    return BLACK if player is WHITE else WHITE

def find_bracket(square, player, board, direction):
    """
    Find a square that forms a bracket with `square` for `player` in the given
    `direction`. Returns None if no such squares exists.
    """
    bracket = square + direction
    if board[bracket] == player:
        return None
    opp = opponent(player)
    while board[bracket] == opp:
        bracket += direction
    return None if board[bracket] in (OUTER, EMPTY) else bracket

def is_legal(move, player, board):
    """Is this a legal move for the player?"""
    hasbracket = lambda direction: find_bracket(move, player, board, direction)
    return board[move] == EMPTY and any(map(hasbracket, DIRECTIONS))

### Making Moves

When the player makes a move, we need to update the board and flip all the bracketed pieces.

In [5]:
def make_move(move, player, board):
    """Update the board to reflect the move by the specified player."""
    board[move] = player
    for d in DIRECTIONS:
        make_flips(move, player, board, d)
    return board

def make_flips(move, player, board, direction):
    """Flip pieces in the given direction as a result of the move by player."""
    bracket = find_bracket(move, player, board, direction)
    if not bracket:
        return
    square = move + direction
    while square != bracket:
        board[square] = player
        square += direction

### Monitoring Players

In [6]:
class IllegalMoveError(Exception):
    def __init__(self, player, move, board):
        self.player = player
        self.move = move
        self.board = board
        
    def __str__(self):
        return '{0} cannot move to square {1}'.format(PLAYERS[self.player], self.move)
    
def legal_moves(player, board):
    """Get a list of all legal moves for player."""
    return [sq for sq in squares() if is_legal(sq, player, board)]

def any_legal_move(player, board):
    """Can player make any moves?"""
    return any(is_legal(sq, player, board) for sq in squares())

### Putting it all together

Each round consists of:
-  Get a move from the current player.
-  Apply it to the board.
-  Switch players. If the game is over, get the final score.

In [7]:
move_number = 1 # The number of the move to be played

def play(black_strategy, white_strategy):
    """Play a game of Othello and return the final board and score."""
    board = initial_board()
    player = BLACK
    strategy = lambda who: black_strategy if who == BLACK else white_strategy
    move_number = 1
    while player is not None:
        move = get_move(strategy(player), player, board)
        make_move(move, player, board)
        player = next_player(board, player)
        move_number += 1
    return board, score(BLACK, board)

def next_player(board, prev_player):
    """Which player should move next? Returns None if no legal moves exist."""
    opp = opponent(prev_player)
    if any_legal_move(opp, board):
        return opp
    elif any_legal_move(prev_player, board):
        return prev_player
    return None

def get_move(strategy, player, board):
    """Call strategy(player, board) to get a move."""
    copy = board[:] # copy the board to prevent cheating
    move = strategy(player, copy)
    if not is_valid(move) or not is_legal(move, player, board):
        raise IllegalMoveError(player, move, copy)
    return move

def get_score(player, board):
    """Compute player's score (number of player's pieces minus opponent's)."""
    mine, theirs = 0, 0
    opp = opponent(player)
    for sq in squares():
        piece = board[sq]
        if piece == player:
            mine += 1
        elif piece == opp:
            theirs += 1
    return mine - theirs

<a id="strategies"></a>

## Play Strategies

### Random

The easiest strategy to implement: simply pick a move at random.

In [8]:
import random

def random_strategy(player, board):
    """A strategy that always chooses a random legal move."""
    return random.choice(legal_moves(player, board))

<a id="localmax"></a>

### Local Maximization

A more sophisticated strategy could look at every available move and evaluate them in some way. This consists of getting a list of legal moves, applying each one to a copy of the board, and choosing the move that results in the "best" board.

In [9]:
def maximizer(evaluate):
    """
    Construct a strategy that chooses the best move by maximizing
    evaluate(player, board) over all boards resulting from legal moves.
    """
    def strategy(player, board):
        def score_move(move):
            return evaluate(player, make_move(move, player, board[:]))
        return max(legal_moves(player, board), key=score_move)
    return strategy

One possible evaluation function is `score`. A strategy constructed with `maximizer(score)` will always make the move that results in the largest immediate gain in pieces.

A more advanced evaluation function might consider the relative worth of each square on the board and weight the score by the value of the pieces held by each player. Since corners and (most) edge squares are very valuable, we could weight those more heavily, and add negative weights to the squares that, if acquired, could lead to the opponent capturing the corners or edges.

<img src='files/img/weighted.png'>

In [10]:
SQUARE_WEIGHTS = [
    0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
    0, 120, -20,  20,   5,   5,  20, -20, 120,   0,
    0, -20, -40,  -5,  -5,  -5,  -5, -40, -20,   0,
    0,  20,  -5,  15,   3,   3,  15,  -5,  20,   0,
    0,   5,  -5,   3,   3,   3,   3,  -5,   5,   0,
    0,   5,  -5,   3,   3,   3,   3,  -5,   5,   0,
    0,  20,  -5,  15,   3,   3,  15,  -5,  20,   0,
    0, -20, -40,  -5,  -5,  -5,  -5, -40, -20,   0,
    0, 120, -20,  20,   5,   5,  20, -20, 120,   0,
    0,   0,   0,   0,   0,   0,   0,   0,   0,   0
]

A strategy constructed as `maximizer(weighted_score)`, then will always return the move that results in the largest immediate "weighted" gain in pieces.

In [11]:
def weighted_score(player, board):
    """
    Compute the difference between the sum of the weights of player's
    squares and the sum of the weights of opponent's squares.
    """
    opp = opponent(player)
    total = 0
    for sq in squares():
        if board[sq] == player:
            total += SQUARE_WEIGHTS[sq]
        elif board[sq] == opp:
            total -= SQUARE_WEIGHTS[sq]
    return total

This is a greedy strategy however, and will result in a local optimum, as the accepted optimal strategy for Othello is to centralize and maximize mobility (the amount of legal moves available to you) by minimizing your pieces and restricting the opponent's mobility.

<img src='files/img/greedy.png'>

We could construct an evaluation function `mobility`. A strategy constructed with `maximizer(mobility)` will always make the move that results in the largest immediate gain in mobility.

We define _current mobility_ as the number of legal moves available to a player, and _potential mobility_ as the number of blank squares that are adjacent to opponent's pieces (called _frontier discs_). These include the legal moves. A better measure of mobility would try to count only good moves. The following function computes both current and potential mobility for a player:

In [12]:
def adj(square):
    """List all the neighbors of a square."""
    return [square + d for d in DIRECTIONS]

def mobility(player, board):
    """
    Current mobility is the number of legal moves.
    Potential mobility is the number of blank squares
    adjacent to an opponent that are not legal moves.
    Returns current and potential mobility for player.
    """
    opp = opponent(player)
    current, potential = 0, 0
    for sq in squares():
        if board[sq] == EMPTY:
            if sq in legal_moves(player, board):
                current += 1
            elif any(board[neighbor] == opp for neighbor in adj[sq]):
                potential += 1
    return current, (current + potential)

<a id="minimax"></a>

### Minimax search

The maximizer strategies are very short-sighted, and a player who can consider the implications of a move several turns in advance could have a significant advantage. We can improve the strategy by searching ahead. Instead of choosing the move that leads immediately to the highest score, we can also consider the opponent's possible replies, our replies to those replies, and so on. By searching through several levels of moves, we can steer away from potential disaster and find good moves that were not immediately apparent.

<img src='files/img/minimax.png'>

In [13]:
def minimax(player, board, depth, evaluate):
    """
    Find the best legal move for player, searching to the specified depth.
    Returns a tuple (min_score, move), where min_score is the guaranteed minimum
    score achievable for player if the move is made.
    """
    
    # We define the value of a board to be the opposite of its value to our
    # opponent, computed by recursively applying `minimax` for our opponent.
    def value(board):
        return -minimax(opponent(player), board, depth-1, evaluate)[0]
    
    # When depth is zero, don't examine possible moves. Just determine the value
    # of this board to the player.
    if depth == 0:
        return evaluate(player, board), None
    
    # We want to evaluate all the legal moves by considering their implications
    # `depth` turns in advance. First, find all the legal moves.
    moves = legal_moves(player, board)
    
    # If player has no legal moves, then either:
    if not moves:
        # the game is over, so the best achievable score is victory or defeat
        if not any_legal_move(opponent(player), board):
            return final_value(player, board), None
        # or we have to pass this turn, so just find the value of this board.
        return value(board), None
    
    # When there are multiple legal moves available, choose the best one by
    # maximizing the value of the resulting boards.
    return max((value(make_move(m, player, board[:])), m) for m in moves)

# Values for endgame boards are big constants
MAX_VALUE = float('inf')
MIN_VALUE = float('-inf')

def final_value(player, board):
    """The game is over. Find the value of this board to player."""
    diff = score(player, board)
    if diff < 0:
        return MIN_VALUE
    elif diff > 0:
        return MAX_VALUE
    return diff

def minimax_searcher(depth, evaluate):
    """
    Construct a strategy that uses `minimax` with the specified leaf board
    evaluation function.
    """
    def strategy(player, board):
        return minimax(player, board, depth, evaluate)[1]
    return strategy

<a id="alphabeta"></a>

### Alpha-Beta Search

Minimax is very effective, but it does too much work: it evaluates many search trees that should be ignored.

Consider what happens when minimax is evaluating two moves, M1 and M2, on one level of a search tree. Suppose minimax determines that M1 can result in a score of S. While evaluating M2, if minimax finds a move in its subtree that could result in a better score than S, the algorithm should immediately quit evaluating M2: the opponent will force us to play M1 to avoid the higher score resulting from M2, so we shouldn't waste time determining just how much better M2 is than M1.

<img src='files/img/alphabeta.png'>

We need to keep track of two values:
-  __alpha:__ the maximum score achievable by any of the moves we have encountered.
-  __beta:__ the score that the opponent can keep us under by playing other moves.

When the algorithm begins, alpha is the smallest value and beta is the largest value. During evaluation, if we find a move that causes `alpha >= beta`, then we can quit searching this subtree since the opponent can prevent us from playing it.

In [14]:
def alphabeta(player, board, alpha, beta, depth, evaluate):
    """
    Find the best legal move for player, searching to the specified depth.
    Like minimax but uses the bounds alpha and beta to prune branches.
    """
    if depth == 0:
        return evaluate(player, board), None
    
    def value(board, alpha, beta):
        # Like in `minimax`, the value of a board is the opposite of its value
        # to the opponent. We pass in `-beta` and `-alpha` as the alpha and
        # beta values, respectively, for the opponent, since `alpha` represents
        # the best score we know we can achieve and is therefore the worst score
        # achievable by the opponent. Similary, `beta` is the worst score that
        # our opponent can hold us to, so it is the best score that they can
        # achieve.
        return -alphabeta(opponent(player), board, -beta, -alpha, depth-1, evaluate)[0]
    
    moves = legal_moves(player, board)
    if not moves:
        if not any_legal_move(opponent(player), board):
            return final_value(player, board), None
        return value(board, alpha, beta), None
    
    best_move = moves[0]
    for move in moves:
        if alpha >= beta:
            # If one of the legal moves leads to a better score than beta, then
            # the opponent will avoid this branch, so we can quit looking.
            break
        val = value(make_move(move, player, board[:]), alpha, beta)
        if val > alpha:
            # If one of the moves leads to a better score than the current best
            # achievable score, then replace it with this one.
            alpha = val
            best_move = move
    return alpha, best_move

def alphabeta_searcher(depth, evaluate):
    def strategy(player, board):
        return alphabeta(player, board, MIN_VALUE, MAX_VALUE, depth, evaluate)[1]
    return strategy

### Alpha-Beta v2.0 - Move Ordering

The alpha-beta cutoffs work when we have established a good move and another move proves to be not as good. Thus, we will be able to make cutoffs earlier if we ensure that good moves are considered first. Our current algorithm loops through the list of `legal_moves`, but `legal_moves` makes no attempt to order the moves in any way. We will call this the _random-ordering_ strategy (even though the ordering is not random at all, square 11 is always considered first, then 12, etc.).

One way to try to generate good moves first is to search highly weighted squares first. Since `legal_moves` considers squares in the order defined by `squares`, all we have to do is redefine the list `squares`.

In [15]:
def squares():
    """
    List all the valid squares on the board
    sorted from highest to lowest weight.
    """
    return sorted([i for i in range(11, 89) if 1 <= (i % 10) <= 8], key=lambda sq: SQUARE_WEIGHTS[sq], reverse=True)

Now the corner squares will automatically be considered first, followed by the other highly weighted squares. We call this the _static-ordering_ strategy, because the ordering is not random, but it does not change depending on the situation.

A more informed way to try to generate good moves first is to sort the moves according to the evaluation function. This means making more evaluations. Previously, only the boards at the leaves of the search tree were evaluated. Now we need to evaluate every board. In order to avoid evaluating a board more than once, we make up a structure called a `node`, which holds a board, the square that was taken to result in that board, and the evaluation value of that board.

```
node = [square, board, value]
```

The search is the same except that nodes are passed around instead of boards, and the nodes are sorted by their value.

In [16]:
def alphabeta2(player, node, alpha, beta, depth, evaluate):
    """
    Alphabeta search, sorting moves by evaluation function
    """
    if depth == 0:
        return node[2], node

    def value(node, alpha, beta):
        return -alphabeta2(opponent(player), negate_value(node), -beta, -alpha, depth-1, evaluate)[0]
    
    board = node[1]
    nodes = legal_nodes(player, board, evaluate)
    
    if not nodes:
        if not any_legal_move(opponent(player), board):
            return final_value(player, board), None
        return value(node, alpha, beta), None
    
    best_node = nodes[0]
    for move in nodes:
        if alpha >= beta:
            # If one of the legal moves leads to a better score than beta, then
            # the opponent will avoid this branch, so we can quit looking.
            break
        val = value(move, alpha, beta)
        if val > alpha:
            # If one of the moves leads to a better score than the current best
            # achievable score, then replace it with this one.
            alpha = val
            best_node = move
    return alpha, best_node

def negate_value(node):
    """Set the value of a node to its negative."""
    node[2] = -node[2]
    return node

def legal_nodes(player, board, evaluate):
    """Return a list of legal moves, each one packed into a node."""
    def fn(move):
        new_board = make_move(move, player, board[:])
        return [move, new_board, evaluate(player, new_board)]
    moves = legal_moves(player, board)
    nodes = sorted([fn(move) for move in moves], key=lambda node: node[2], reverse=True)
    return nodes
    
def alphabeta_searcher2(depth, evaluate):
    """Return a strategy that does Alphabeta search with sorted moves."""
    def strategy(player, board):
        node = [None, board, evaluate(player, board)]
        return alphabeta2(player, node, MIN_VALUE, MAX_VALUE, depth, evaluate)[1][0]
    return strategy

### Alpha-Beta v3.0 - Killer Instinct

We saw before that the search routines look at tens of thousands of boards per move. Currently, each board position is created anew by `board[:]` and discarded soon thereafter. We could avoid generating all this garbage by reusing the same board at each depth. We'd still need to keep the board from the previous depth for use when the search backs up. Thus, a vector of boards is needed. In the following we assume that we will never search deeper than 40 depth. This is a safe assumption, as even the fastest Othello programs can only search about 15 depth before running out of time.

In [17]:
ply_boards = [initial_board() for _ in range(40)]

In the previous section, we considered to possibility of searching moves in a different order, in an attempt to search the better moves first, thereby getting more alpha-beta pruning. In this section, we consider the _killer heuristic_, which states that a move that has proven to be a good one in one line of play is also likely to be a good one in another line of play. To use chess as perhaps a more familiar example, suppose I consider one move, and it leads to the opponent replying by capturing my queen. This is a killer move, one that I would like to avoid. Therefore, when I consider other possible moves, I want to immediately consider the possibility of the opponent making that queen-capturing move.

The function `alphabeta3` adds the parameter `killer`, which is the best move found so far at the current level. After we determine the `legal_moves`, we use `put_first` to put the killer move first, if it is in fact a legal move. When it comes time to search the next level, we keep track of the best move in `killer2`. This requires keeping track of the value of the best move in `killer2_val`. Everything else is unchanged, except that we get a new board by recycling the `ply_boards` vector rather than by allocating fresh ones.

In [18]:
def alphabeta3(player, board, alpha, beta, depth, evaluate, killer):
    """
    Alphabeta search, putting killer move first.
    """
    if depth == 0:
        return evaluate(player, board), None
    
    def value(board, alpha, beta, killer):
        val, reply = alphabeta3(opponent(player), board, -beta, -alpha, depth-1, evaluate, killer)
        return -val, reply
    
    moves = put_first(killer, legal_moves(player, board))
    if not moves:
        if not any_legal_move(opponent(player), board):
            return final_value(player, board), None
        return value(board, alpha, beta, None)[0], None
    
    best_move = moves[0]
    new_board = ply_boards[depth]
    killer2 = None
    killer2_val = MAX_VALUE
    for move in moves:
        if alpha >= beta:
            # If one of the legal moves leads to a better score than beta, then
            # the opponent will avoid this branch, so we can quit looking.
            break
        val, reply = value(make_move(move, player, replace(new_board, board)), alpha, beta, killer2)
        if val > alpha:
            # If one of the moves leads to a better score than the current best
            # achievable score, then replace it with this one.
            alpha = val
            best_move = move
        if reply is not None and val < killer2_val:
            # If one of the moves leads to a reply that is worse than our worst
            # case scenario killer2, then replace it with this one.
            killer2 = reply
            killer2_val = val
    return alpha, best_move

def put_first(killer, moves):
    """
    Move the killer move to the front of moves,
    if the killer move is in fact a legal move.
    """
    if killer in moves:
        moves.insert(0, moves.pop(moves.index(killer)))
    return moves

def replace(seq1, seq2):
    """Copies one sequence into another"""
    seq1[:] = seq2
    return seq1
    
def alphabeta_searcher3(depth, evaluate):
    """Return a strategy that does Alphabeta search with killer moves"""
    def strategy(player, board):
        return alphabeta3(player, board, MIN_VALUE, MAX_VALUE, depth, evaluate, None)[1]
    return strategy

## Championship Programs: Iago and Bill
As mentioned in the introduction, the unpredictability of Othello makes it a difficult game for humans to master, and thus programs that search deeply can do comparatively well. In fact, in 1981 the reigning champion, Jonathan Cerf, proclaimed "In my opinion the top programs ... are now equal (if not superior) to the best human players." In discussing Rosenbloom's Iago program (1982), Cerf went on to say "I understand Paul Rosenbloom is interested in arranging a match against me. Unfortunately my schedule is very full, and I'm going to see that it remains that way for the foreseeable future."

In 1989, another program, Bill (Lee and Mahajan 1990) beat the highest rated American Othello player, Brian Rose, by a score of 56-8. Bill's evaluation function is fast enough to search 6-8 depths under tournament conditions, yet it is so accurate that it beats its creator, Kai-Fu Lee, searching only 1 depth. (However, Lee is only a novice Othello player; his real interest is in speech recognition; see Waibel and Lee 1991.) There are other programs that also play at a high level, but they have not been written up in the AI literature as Iago and Bill have.

In this section we present an evaluation function based on Iago's, although it also contains elements of Bill, and of an evaluation function written by Eric Wefald in 1989. The evaluation function makes use of two main features: _mobility_ and _edge stability_.

### Edge Stability

Success at Othello often hinges around edge play, and both Iago and Bill evaluate the edges carefully. Edge analysis is made easier by the fact that the edges are fairly independent of the interior of the board: once a piece is placed on the edge, no interior moves can flip it. This independence allows a simplifying assumption: to evaluate a position's edge strength, evaluate each of the four edges independently, without consideration of the interior of the board. The evaluation can be made more accurate by considering the X-squares to be part of the edge.

Even evaluating a single edge is a time-consuming task, so Bill and Iago compile away the evaluation by building a table of all possible edge positions. An "edge" according to Bill is ten squares: the eight actual edge squares and the two X-squares. Since each square can be black, white, or empty, there are 310 or 59,049 possible edge positions-a large but manageable number.

The value of each edge position is determined by a process of successive approximation. Just as in a minimax search, we will need a static edge evaluation function to determine the value of an edge position without search. This static edge evaluation function is applied to every possible edge position, and the results are stored in a 59,049 element vector. The static evaluation is just a weighted sum of the occupied squares, with different weights given depending on if the piece is stable or unstable.

Each edge position's evaluation can be improved by a process of search. Iago uses a single depth search: given a position, consider all moves that could be made (including no move at all). Some moves will be clearly legal, because they flip pieces on the edge, but other moves will only be legal if there are pieces in the interior of the board to flip. Since we are only considering the edge, we don't know for sure if these moves are legal. They will be assigned probabilities of legality. The updated evaluation of a position is determined by the values and probabilities of each move. This is done by sorting the moves by value and then summing the product of the value times the probability that the move can be made. This process of iterative approximation is repeated five times for each position. At that point, Rosenbloom reports, the values have nearly converged.

In effect, this extends the depth of the normal alpha-beta search by including an edge-only search in the evaluation function. Since each edge position with _n_ pieces is evaluated as a function of the positions with _n + 1_ pieces, the search is complete - it is an implicit 10-depth search.

Calculating edge stability is a bit more complicated than the other features. The first step is to define a variable, `edge_table`, which will hold the evaluation of each edge position, and a constant, `edge_and_x_lists`, which is a list of the squares on each of the four edges. Each edge has ten squares because the X-squares are included.

In [19]:
# Array of values for edge positions
edge_table = [0 for _ in range(3**10)]

# The four edges (with their X-squares)/
edge_and_x_lists = [[22, 11, 12, 13, 14, 15, 16, 17, 18, 27],
                    [72, 81, 82, 83, 84, 85, 86, 87, 88, 77],
                    [22, 11, 21, 31, 41, 51, 61, 71, 81, 72],
                    [27, 18, 28, 38, 48, 58, 68, 78, 88, 77]]

Now for each edge we can compute an index into the edge table by building a 10-digit base-3 number, where each digit is 1 if the corresponding edge square is occupied by the player, 2 if by the opponent, and 0 if empty. The function `edge_index` computes this, and `edge_stability` sums the values of the four edge indexes.

In [20]:
def edge_index(player, board, squares):
    """The index counts 1 for player; 2 for opponent,
    on each square -- summed as a base 3 number."""
    opp = opponent(player)
    index = 0
    for sq in squares:
        index *= 3
        if board[sq] == player:
            index += 1
        elif board[sq] == opp:
            index += 2
    return index

def edge_stability(player, board):
    """Total edge evaluation for player"""
    score = sum(edge_table[edge_index(player, board, edge)] for edge in edge_and_x_lists)
    return score

The function `edge_stability` is all we will need in Iago's evaluation function, but we still need to generate the edge table. Since this needs to be done only once, we don't have to worry about efficiency. In particular, rather than invent a new data structure to represent edges, we will continue to use complete boards, even though they will be mostly empty. The computations for the edge table will be made on the top edge, from the point of view of black, with black to play. But the same table can be used for white, or for one of the other edges, because of the way the edge index is computed.

Each position in the table is first initialized to a static value computed by a kind of weighted-squares metric, but with different weights depending on if a piece is in danger of being captured. After that, each position is updated by considering the possible moves that can be made from the position, and the values of each of these moves.

In [21]:
top_edge = edge_and_x_lists[0]

def init_edge_table():
    """Initialize `edge_table`, starting from the empty board."""
    # Initialize the static values
    for n_pieces in range(11):
        def fn(board, index):
            edge_table[index] = static_edge_stability(BLACK, board)
        map_edge_n_pieces(fn, BLACK, initial_board(), n_pieces, top_edge, 0)
    # Now iterate five times trying to improve
    for _ in range(5):
        for n_pieces in range(9, 0, -1):
            def fn(board, index):
                edge_table[index] = possible_edge_moves_value(BLACK, board, index)
            map_edge_n_pieces(fn, BLACK, initial_board(), n_pieces, top_edge, 0)

The function `map_edge_n_pieces` iterates through all edge positions with a total of `n` pieces (of either color), applying a function to each such position. It also keeps a running count of the edge index as it goes. The function should accept two arguments: the board and the index. Note that a single board can be used for all the positions because squares are reset after they are used. The function has three cases: if the number of squares remaining is less than `n`, then it will be impossible to place `n` pieces on those squares, so we give up. If there are no more squares then `n` must also be zero, so this is a valid position, and the function `fn` is called. Otherwise we first try leaving the current square blank, then try filling it with player's piece, and then with the opponent's piece, in each case calling `map_edge_n_pieces` recursively.

In [22]:
def map_edge_n_pieces(fn, player, board, n, squares, index):
    """
    Call fn on all edges with n pieces.
    Index counts 1 for player, 2 for opponent
    """
    if len(squares) < n:
        return
    elif not squares:
        fn(board, index)
    else:
        index3 = index * 3
        sq = squares[0]
        map_edge_n_pieces(fn, player, board, n, squares[1:], index3)
        if n > 0 and board[sq] == EMPTY:
            board[sq] = player
            map_edge_n_pieces(fn, player, board, n-1, squares[1:], index3+1)
            board[sq] = opponent(player)
            map_edge_n_pieces(fn, player, board, n-1, squares[1:], index3+2)
            board[sq] = EMPTY

The function `possible_edge_moves_value` searches through all possible moves to determine an edge value that is more accurate than a static evaluation. It loops through every empty square on the edge, calling `possible_edge_move` to return a (_probability value_) pair. Since it is also possible for a player not to make any move at all on an edge, the pair (`1.0`, _current value_) is also included.

In [23]:
def possible_edge_moves_value(player, board, index):
    """
    Consider all possible edge moves.
    Combine their values into a single number.
    """
    x = [(1.0, edge_table[index])]
    y = [possible_edge_move(player, board, sq) for sq in top_edge if board[sq] == EMPTY]
    possibilities = x + y
    return combine_edge_moves(possibilities, player)

The value of each position is determined by making the move on the board, then looking up in the table the value of the resulting position for the opponent, and negating it (since we are interested in the value to us, not to our opponent).

In [24]:
def possible_edge_move(player, board, sq):
    """Return a (prob, val) pair for a possible edge move."""
    num_player = 1 if player == BLACK else 2
    new_board = replace(ply_boards[num_player], board)
    make_move(sq, player, new_board)
    prob = edge_move_probability(player, board, sq)
    val = -edge_table[edge_index(opponent(player), new_board, top_edge)]
    return (prob, val)

The possible moves are combined with `combine_edge_moves`, which sorts the moves best-first. (Since `init_edge_table` started from black's perspective, black tries to maximize and white tries to minimize scores.) We then go down the moves, increasing the total value by the value of each move times the probability of the move, and decreasing the remaining probability by the probability of the move. Since there will always be at least one move (pass) with probability 1.0, this is guaranteed to converge. In the end, we round off the total value, so that we can do the run-time calculations with integers.

In [25]:
def combine_edge_moves(possibilities, player):
    """Combine the best moves."""
    prob = 1.0
    val = 0.0
    fn = True if player == BLACK else False
    for pair in sorted(possibilities, key=lambda x: x[1], reverse=fn):
        if prob < 0.0:
            break
        val += prob * pair[0] * pair[1]
        prob -= prob * pair[0]
    return round(val)

We still need to compute the probability that each possible edge move is legal. These probabilities should reflect things such as the fact that it is easy to capture a corner if the opponent is in the adjacent X-square, and very difficult otherwise. First we define some functions to recognize corner and X-squares and relate them to their neighbors:

In [26]:
corner_xsqs = [(11, 22), (18, 27), (81, 72), (88, 77)]

def corner_p(sq):
    """Return the tuple which contains the corner square"""
    return next((x for x in corner_xsqs if x[0] == sq), None)

def x_square_p(sq):
    """Return the tuple which contains the x-square"""
    return next((x for x in corner_xsqs if x[1] == sq), None)

def x_square_for(corner):
    """Return the x-square for a corner square"""
    tuple = [x for x in corner_xsqs if x[0] == corner]
    return tuple[0][1] if tuple else None

def corner_for(xsq):
    """Return the corner square for an x-square"""
    tuple = [x for x in corner_xsqs if x[1] == xsq]
    return tuple[0][0] if tuple else None

Now we consider the probabilities. There are four cases. First, since we don't know anything about the interior of the board, we assume each player has a 50% chance of being able to play in an X-square. Second, if we can show that a move is legal (because it flips opponent pieces on the edge) then it has 100% probability. Third, for the corner squares, we assign a 90% chance if the opponent occupies the X-square, 10% if it is empty and only .1% if we occupy it. Otherwise, the probability is determined by the two neighboring squares: if a square is next to one or more opponents, it is more likely we can move there; if it is next to our pieces it is less likely. If it is legal for the opponent to move into the square, then the chances are cut in half (although we may still be able to move there, since we move first).

In [27]:
def edge_move_probability(player, board, square):
    """What's the probability that player can move to this square?"""
    # X-squares
    if x_square_p(square):
        return 0.5
    # Immediate capture
    elif is_legal(square, player, board):
        return 1.0
    # Move to corner depends on X-square
    elif corner_p(square):
        x_square = x_square_for(square)
        if board[x_square] == EMPTY:
            return 0.1
        elif board[x_square] == player:
            return 0.001
        else:
            return 0.9
    else:
        val = [[.1, .4, .7],
               [.05, .3, None],
               [.01, None, None]]
        x = count_edge_neighbors(player, board, square)
        y = count_edge_neighbors(opponent(player), board, square)
        if is_legal(square, opponent(player), board):
            return val[x][y]/2
        else:
            return val[x][y]
        
def count_edge_neighbors(player, board, square):
    """Count the neighbors of this square occupied by player."""
    inc = [1, -1]
    neighbors = [board[square+i] for i in inc]
    return sum(1 for x in neighbors if x == player)

Now we return to the problem of determining the static value of an edge position. This is computed by a weighted-squares metric, but the weights depend on the _stability_ of each piece. A piece is called stable if it cannot be captured, unstable if it is in immediate danger of being captured, and semistable otherwise. A table of weights follows for each edge square and stability. Note that corner squares are always stable, and X-squares we will call semistable if the adjacent corner is taken, and unstable otherwise.

In [28]:
#                     stab  semi  un
static_edge_table = [[None, 0, -2000],  # X
                     [700, None, None], # corner
                     [1200, 200, -25],  # C
                     [1000, 200, 75],   # A
                     [1000, 200, 50],   # B
                     [1000, 200, 50],   # B
                     [1000, 200, 75],   # A
                     [1200, 200, -25],  # C
                     [700, None, None], # corner
                     [None, 0, -2000]]  # X

The static evaluation then just sums each piece's value according to this table:

In [29]:
def static_edge_stability(player, board):
    """Compute this edge's static stability."""
    score = 0
    for i in range(len(top_edge)):
        sq = top_edge[i]
        if board[sq] == EMPTY:
            score += 0
        elif board[sq] == player:
            score += static_edge_table[i][piece_stability(board, sq)]
        else:
            score -= static_edge_table[i][piece_stability(board, sq)]
    return score

The computation of stability is fairly complex. It centers around finding the two "pieces," `p1` and `p2`, which lay on either side of the piece in question and which are not of the same color as the piece. These "pieces" may be empty, or they may be off the board. A piece is unstable if one of the two is empty and the other is the opponent; it is semistable if there are opponents on both sides and at least one empty square to play on, or if it is surrounded by empty pieces. Finally, if either `p1` or `p2` is nil then the piece is stable, since it must be connected by a solid wall of pieces to the corner.

In [30]:
def piece_stability(board, sq):
    """Evaluate whether a square is stable, unstable or semi-stable"""
    stable, semi_stable, unstable = 0, 1, 2
    if corner_p(sq):
        return stable
    elif x_square_p(sq):
        if board[corner_for(sq)] == EMPTY:
            return unstable
        else:
            return semi_stable
    else:
        player = board[sq]
        opp = opponent(player)
        p1 = next(p for p in board[sq:20] if p != player)
        p2 = next(p for p in board[sq-1:9:-1] if p != player)
        
        # Unstable pieces can be captured immediately
        # by playing in the empty square
        if (p1 == EMPTY and p2 == opp) or (p2 == EMPTY and p1 == opp):
            return unstable
        
        # Semi-stable pieces might be captured
        elif p1 == opp and p2 == opp and next((p for p in board[11:19] if p == EMPTY), None):
            return semi_stable
        elif p1 == EMPTY and p2 == EMPTY:
            return semi_stable
        
        # Stable pieces can never be captured
        else:
            return stable

The edge table can now be built by a call to `init_edge_table`. After the table is built once, it is a good idea to save it so that we won't need to repeat the initialization.

In [31]:
# Save data into a file
def save_data(filename, data):
    with open(filename, 'w') as f:
        f.write(' '.join(str(x) for x in data))

# Load data from file
def load_data(filename):
    with open(filename) as f:
        dataset = [int(x) for x in next(f).split()]
    return dataset

init_edge_table()
save_data('edge_table.txt', edge_table)

### Combining the Factors

Now we have a measure of the three factors: current mobility, potential mobility and edge stability. All that remains is to find a good way to combine them into a single evaluation metric. The combination function used by Rosenbloom (1982) is a linear combination of the three factors, but each factor's coefficient is dependent on the move number. Rosenbloom's features are normalized to the range [-1000, 1000]; we normalize to the range [-1, 1] by doing a division after multiplying by the coefficient. That allows us to use integers for the coefficients. Since our three factors are not calculated in quite the same way as Rosenbloom's, it is not surprising that his coefficients are not the best for our program. The edge coefficient was doubled and the potential coefficient cut by a factor of five.

In [32]:
def Iago_eval(player, board):
    """
    Combine edge stability, current mobility and
    potential mobility to arrive at an evaluation.
    """
    # The three factors are multiplied by coefficients
    # that vary by move number
    c_edg = 312000 + 6240 * move_number
    if move_number < 25:
        c_cur = 50000 + 2000 * move_number
    else:
        c_cur = 75000 + 1000 * move_number
    c_pot = 20000
    
    p_cur, p_pot = mobility(player, board)
    o_cur, o_pot = mobility(opponent(player), board)
    
    score1 = round(c_edge * edge_stability(player, board) / 32000)
    score2 = round(c_cur * (p_cur - o_cur) / (p_cur + o_cur + 2))
    score3 = round(c_pot * (p_pot - o_pot) / (p_pot + o_pot + 2))
    
    score = score1 + score2 + score3
    return score